In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
FOLDERNAME = "cs231n/assignments/assignment2/"
assert FOLDERNAME is not None

In [3]:
import sys
sys.path.append("/content/drive/My Drive/{}".format(FOLDERNAME))

In [4]:
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/My Drive/cs231n/assignments/assignment2/cs231n/datasets
/content/drive/My Drive/cs231n/assignments/assignment2


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np
import torch.nn.functional as F

USE_GPU = True
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print_every = 100

print("using device:", device)

using device: cuda


In [6]:
NUM_TRAIN = 49000

transform = T.Compose([T.ToTensor(),
                       T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

cifar10_train = dset.CIFAR10("./cs231n/datasets", train=True, download=True, transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10("./cs231n/datasets", train=True, download=True, transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10("./cs231n/datasets", train = False, download = True, transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
class FourLayerConvNet(nn.Module):
  def __init__(self, in_channel, channel_1, channel_2, hidden_dim, num_classes):
    super().__init__()

    self.conv_1 = nn.Conv2d(in_channel, channel_1, kernel_size=5, padding=2)
    self.bn1 = nn.BatchNorm2d(channel_1)
    self.relu_1 = nn.ReLU()
    nn.init.kaiming_normal_(self.conv_1.weight)

    self.conv_2 = nn.Conv2d(channel_1, channel_2, kernel_size=3, padding=1)
    self.bn2 = nn.BatchNorm2d(channel_2)
    self.relu_2 = nn.ReLU()
    nn.init.kaiming_normal_(self.conv_2.weight)

    self.fc_1 = nn.Linear(channel_2*32*32, hidden_dim)
    self.flat = nn.Flatten()
    self.fc_bn = nn.BatchNorm1d(hidden_dim)
    self.relu1 = nn.ReLU()
    nn.init.kaiming_normal_(self.fc_1.weight)

    self.fc_2 = nn.Linear(hidden_dim, num_classes)
    nn.init.kaiming_normal_(self.fc_2.weight)

  def forward(self, x):

    x = self.conv_1(x)
    x = self.bn1(x)
    x = self.relu_1(x)
    x = self.conv_2(x)
    x = self.bn2(x)
    x = self.relu_2(x)
    x = self.flat(x)
    x = self.fc_1(x)
    x = self.fc_bn(x)
    x = self.relu1(x)
    scores = self.fc_2(x)
    return scores


In [8]:
def modeltest():
  model = FourLayerConvNet(3, 12, 8, 100, 10)
  x = torch.zeros((64, 3, 32, 32), dtype=dtype)
  scores = model(x)
  print(scores.size())

modeltest()

torch.Size([64, 10])


In [14]:
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on validation set")
  else:
    print("Checking accuracy on test set")
  num_correct, num_samples = 0, 0
  model.to(device=device)
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device, dtype=dtype)
      y = y.to(device=device, dtype=torch.long)
      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (y==preds).sum()
      num_samples += preds.size(0)
    acc = float(num_correct/num_samples)
    print(("Got %d / %d correct (%.2f)") % (num_correct, num_samples, acc*100))
  return acc

In [16]:
check_accuracy(loader_val, FourLayerConvNet(3, 12, 8, 10, 10))

Checking accuracy on validation set
Got 98 / 1000 correct (9.80)


0.09800000488758087

In [17]:
def train(model, optimizer, epochs=1):
  model = model.to(device=device)
  for e in range(epochs):
    for t, (x,y) in enumerate(loader_train):
      model.train()
      x = x.to(device=device, dtype=dtype)
      y = y.to(device=device, dtype=torch.long)

      scores = model(x)
      loss = F.cross_entropy(scores, y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if t%print_every == 0:
        print("Iteration %d, loss = %.4f" % (t, loss.item()))
        check_accuracy(loader_val, model)

In [18]:
learning_rate = 3e-3
channel_1 = 64
channel_2 = 32
hidden_dim = 10

In [19]:
model = FourLayerConvNet(3, channel_1, channel_2, hidden_dim, num_classes=10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
train(model, optimizer, epochs=1)

Iteration 0, loss = 2.7906
Checking accuracy on validation set
Got 100 / 1000 correct (10.00)
Iteration 100, loss = 1.9614
Checking accuracy on validation set
Got 282 / 1000 correct (28.20)
Iteration 200, loss = 1.7944
Checking accuracy on validation set
Got 336 / 1000 correct (33.60)
Iteration 300, loss = 1.8418
Checking accuracy on validation set
Got 359 / 1000 correct (35.90)
Iteration 400, loss = 1.7112
Checking accuracy on validation set
Got 386 / 1000 correct (38.60)
Iteration 500, loss = 1.7899
Checking accuracy on validation set
Got 406 / 1000 correct (40.60)
Iteration 600, loss = 1.7574
Checking accuracy on validation set
Got 439 / 1000 correct (43.90)
Iteration 700, loss = 1.6725
Checking accuracy on validation set
Got 437 / 1000 correct (43.70)
